In [2]:
# Importing the libraries we will need throughout the code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as acc
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

RANDOM_SEED = 42

### Importando os DataSets

No momento de construção das features, utilizamos dois computadores diferentes, e salvamos as features em dois arquivos separados (um na pasta Henry e um na pasta Vitor). Por isso, precisaremos juntar essas features em um dataframe único.

Além disso, uma série de features (informações sobre a Transformada de Fourrier) foram colhidas duas vezes. Por isso também se faz necessário remover uma série de informações.

##### Dataset de Treino

In [14]:
vitor_train.columns

Index(['Unnamed: 0', '1Q', '2Q', 'IQR', 'Unnamed: 0.1', 'fpath', 'h1Q', 'h2Q',
       'hIQR', 'hmax', 'hmean', 'hmedian', 'hmin', 'hstd', 'htempo', 'label',
       'max', 'mean', 'median', 'min', 'p1Q', 'p2Q', 'pIQR', 'pmax', 'pmean',
       'pmedian', 'pmin', 'pstd', 'ptempo', 'std', 'tempo'],
      dtype='object')

In [15]:
vitor_train.head()

,Unnamed: 0,1Q,2Q,IQR,Unnamed: 0.1,fpath,h1Q,h2Q,hIQR,hmax,...,p2Q,pIQR,pmax,pmean,pmedian,pmin,pstd,ptempo,std,tempo
0,0,6.230491,27.272478,21.041986,0.0,00044347.wav,1.821884,16.757354,14.935470,494.158194,...,10.835077,7.138589,141.885020,8.332571,6.609736,0.009260,7.121350,120.185320,34.076773,120.185320
1,1,5.674157,35.867089,30.192933,1.0,001ca53d.wav,3.443734,32.837873,29.394139,3168.806360,...,7.231589,5.058523,29.293637,5.162963,4.335608,0.002830,3.867669,97.508844,141.165615,97.508844
2,2,0.012762,0.066496,0.053734,2.0,002d256b.wav,0.002055,0.006511,0.004455,3.861467,...,0.063026,0.051539,3.758787,0.076217,0.027698,0.000198,0.183175,135.999178,0.264248,135.999178
3,3,0.420803,0.977241,0.556438,3.0,0033e230.wav,0.096151,0.205320,0.109168,787.918924,...,0.930007,0.431827,7.764286,0.772548,0.712938,0.000439,0.704018,139.674831,7.775617,135.999178
4,4,0.646084,3.692972,3.046888,4.0,00353774.wav,0.254311,1.683002,1.428691,5412.206944,...,1.716842,1.365045,80.081093,1.535181,0.685752,0.001026,2.317918,117.453835,61.102422,117.453835


In [22]:
vitor_train = pd.read_csv('vitor/working.csv')
vitor_train = vitor_train.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
henry_train = pd.read_csv('henry/train_dataset.csv', header=1)
henry_train = henry_train.rename(columns={'Unnamed: 0': 'fpath'})
n_fpath_vitor = [fpath.replace('audio_train/audio_train/', '') for fpath in vitor_train.fpath]
vitor_train.fpath = n_fpath_vitor
columns_to_drop = ['1Q', '2Q', 'IQR', 'max', 'mean', 'median', 'min', 'std', 'tempo']
vitor_train = vitor_train.drop(columns_to_drop, axis=1)
henry_train = henry_train.drop(['filepath', 'label'], axis=1)

In [26]:
train = vitor_train.merge(henry_train, how='inner', on='fpath')

##### Dataset de Teste

In [32]:
vitor_test = pd.read_csv('vitor/test_info.csv')
vitor_test = vitor_test.drop(['Unnamed: 0'], axis=1)
henry_test = pd.read_csv('henry/test_dataset.csv', header=1)
henry_test = henry_test.rename(columns={'Unnamed: 0': 'fpath'})
n_fpath_vitor = [fpath.replace('../audio_test/', '') for fpath in vitor_test.fpath]
vitor_test.fpath = n_fpath_vitor
columns_to_drop = ['1Q', '2Q', 'IQR', 'max', 'mean', 'median', 'min', 'std', 'tempo']
vitor_test = vitor_test.drop(columns_to_drop, axis=1)
henry_test = henry_test.drop(['filepath', 'label'], axis=1)

In [33]:
test = vitor_test.merge(henry_test, how='inner', on='fpath')

#### Modelo Iniciais

In [39]:
columns_on_df = list(test.columns)
columns_on_df
x_columns = columns_on_df
x_columns.remove('fpath')
x_columns.remove('label')

In [41]:
# Getting Train and Test data
x_train = train[x_columns]
y_train = train['label']
x_test = test[x_columns]
y_test = test['label']

In [43]:
# Feature Scalling the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [52]:
# Creating our Models
knn_model = KNeighborsClassifier(n_neighbors=10, weights='distance')
rf_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED)
gb_model = GradientBoostingClassifier(random_state=RANDOM_SEED)

In [53]:
# Fitting the Models
knn_model.fit(x_train, y_train)
rf_model.fit(x_train, y_train)
gb_model.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False)

In [54]:
# Predicting the Categories
y_knn = knn_model.predict(x_test)
y_rf = rf_model.predict(x_test)
y_gb = gb_model.predict(x_test)

In [56]:
# Getting Accuracy
knn_acc = acc(y_test, y_knn)
rf_acc = acc(y_test, y_rf)
gb_acc = acc(y_test, y_gb)

In [57]:
knn_acc, rf_acc, gb_acc

(0.4, 0.536875, 0.459375)

In [58]:
rf_model.feature_importances_

array([0.01759447, 0.01687904, 0.01734185, 0.02135264, 0.01646443,
       0.01595739, 0.01677577, 0.0184353 , 0.01530938, 0.01764033,
       0.01556924, 0.01652948, 0.02561075, 0.0207645 , 0.01639304,
       0.01511601, 0.0250867 , 0.01662588, 0.01645119, 0.01500338,
       0.01498774, 0.01997226, 0.01404006, 0.01461869, 0.01451377,
       0.0165739 , 0.03012101, 0.0209563 , 0.01931265, 0.02135256,
       0.0228187 , 0.0243321 , 0.03179025, 0.02254543, 0.02997642,
       0.02514545, 0.02254187, 0.01797004, 0.02324065, 0.02841772,
       0.02161909, 0.02268544, 0.02607816, 0.01766764, 0.01654813,
       0.01848837, 0.01824974, 0.02074193, 0.02669386, 0.01909929])